## Segmentation and clustering

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

In [82]:
from bs4 import BeautifulSoup
from requests import get
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = get(url)
import pandas as pd
import numpy as np
html_soup = BeautifulSoup(response.content, 'lxml')
table = html_soup.find('table')
df = pd.read_html(str(table), header=0)[0]
df = df[df.Borough != 'Not assigned']
df = pd.DataFrame(df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join))
df = df.reset_index()
df.Neighbourhood[df.Neighbourhood == 'Not assigned'] = df.Borough
df.rename(columns={'Postcode':'PostalCode'}, inplace=True)
df_geo = pd.read_csv('Geospatial_Coordinates.csv')
df_geo.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
df_res = df.merge(df_geo, how='left', left_on='PostalCode', right_on='PostalCode')
df_res.head(5)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

* to add enough Markdown cells to explain what you decided to do and to report any observations you make.
* to generate maps to visualize your neighborhoods and how they cluster together.

### Mapping
1. Install libraries and import modules

In [70]:
#!pip install geopy
from geopy.geocoders import Nominatim # Convert and address into latitude and longitude values
from pandas.io.json import json_normalize  #Transform json file into pandas dataframe

#Plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Import k-means from clustering
from sklearn.cluster import KMeans

#!pip install folium # Install folium
import folium
print('Imported Libraries')

Imported Libraries


Define a useragent for geocoder

In [71]:
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geolocation of Toronto is {}, {}'.format(latitude, longitude))

The geolocation of Toronto is 43.653963, -79.387207


In [77]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
#Add markers to the map
for lat, lng, borough, neighbourhood in zip(df_res['Latitude'], df_res['Longitude'],
                                           df_res['Borough'],
                                           df_res['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat,lng],
                       radius=5,
                       popup=label,
                       color='blue',
                       fill=True,
                       fill_color='#3186cc',
                       fill_opacity=0.7,
                       parse_html=False).add_to(map_toronto)
map_toronto

Let us select only the neighbourhoods in Downtown Toronto

In [104]:
toronto_data = df_res[df_res['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


In [105]:
address = 'Central Toronto, ON'
geolocator = Nominatim(user_agent='toronto_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geo coordinates for Central Toronto is {}, {}'.format(latitude, longitude))

The geo coordinates for Central Toronto is 43.653963, -79.387207


In [106]:
map_central_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)
#Add markers to the map
for lat, lng, borough, neighbourhood in zip(toronto_data['Latitude'], toronto_data['Longitude'],
                                           toronto_data['Borough'],
                                           toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat,lng],
                       radius=5,
                       popup=label,
                       color='blue',
                       fill=True,
                       fill_color='#3186cc',
                       fill_opacity=0.7,
                       parse_html=False).add_to(map_central_toronto)
map_central_toronto

In [98]:
CLIENT_ID = 'LH2OJQC5DUE5Y3TQZVD3DXDH2A3ASD5MANEVAKEMGDJN0HWT' # Inputs for 4Square Api
CLIENT_SECRET = 'YRM0HCKVDAG5GJU1JYCKPMUWJD0IZUVF1N0ZY3GRKLC3MXT3'
VERSION = '20190930' # Set the version to a date is current

Get the name of the neighbourhood

In [107]:
toronto_data.loc[0, 'Neighbourhood']

'Rosedale'

Get the latitude and longitude values of the neighbourhood

In [108]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rosedale are 43.6795626, -79.37752940000001.


Get the top 100 venues that are in Lawrence Park

In [113]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=LH2OJQC5DUE5Y3TQZVD3DXDH2A3ASD5MANEVAKEMGDJN0HWT&client_secret=YRM0HCKVDAG5GJU1JYCKPMUWJD0IZUVF1N0ZY3GRKLC3MXT3&v=20190930&ll=43.6795626,-79.37752940000001&radius=500&limit=100'

In [114]:
results = get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d985abd531593002c6fa554'},
 'response': {'headerLocation': 'Rosedale',
  'headerFullLocation': 'Rosedale, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.6840626045, 'lng': -79.37131878274371},
   'sw': {'lat': 43.675062595499995, 'lng': -79.38374001725632}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bae2150f964a520df873be3',
       'name': 'Mooredale House',
       'location': {'address': '146 Crescent Rd.',
        'crossStreet': 'btwn. Lamport Ave. and Mt. Pleasant Rd.',
        'lat': 43.678630645646535,
        'lng': -79.38009142511322,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.678630645646535,
          'lng': -79.380091425113

In [115]:
# Function that extracts the catgory of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean the json data and structure it into a dataframe

In [116]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Mooredale House,Building,43.678631,-79.380091
1,Rosedale Park,Playground,43.682328,-79.378934
2,Whitney Park,Park,43.682036,-79.373788
3,Alex Murray Parkette,Park,43.678300,-79.382773
4,Milkman's Lane,Trail,43.676352,-79.373842


In [117]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


Explore the neighbourhoods in Downtown Toronto

In [120]:
#Function to repeat the same process for all the neighbourhoods in Downtown Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [121]:
#Now we get all the venues for Downtown Toronto
downtown_venues = getNearbyVenues(names=toronto_data['Neighbourhood'], 
                                  latitudes=toronto_data['Latitude'], 
                                  longitudes=toronto_data['Longitude'])

Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie


In [122]:
#Check the size of the resulting dataframe
print(downtown_venues.shape)
downtown_venues.head()

(1295, 7)


,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Mooredale House,43.678631,-79.380091,Building
1,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
2,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
3,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
4,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail


In [123]:
# Checking how many venues was returned for each neighbourhood
downtown_venues.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,57,57,57,57,57,57
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",15,15,15,15,15,15
"Cabbagetown, St. James Town",44,44,44,44,44,44
Central Bay Street,85,85,85,85,85,85
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16
Church and Wellesley,86,86,86,86,86,86
"Commerce Court, Victoria Hotel",100,100,100,100,100,100


In [124]:
# Check how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(downtown_venues['Venue Category'].unique())))

There are 206 uniques categories.


## Analyzing each neighbourhood

In [126]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
downtown_onehot['Neighbourhood'] = downtown_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [127]:
# Examine the new dataframe
downtown_onehot.shape

(1295, 207)

In [128]:
#Group the rows by neighbourhood
downtown_grouped = downtown_onehot.groupby('Neighbourhood').mean().reset_index()
downtown_grouped

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.00,...,0.00,0.00,0.0,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00,0.00,0.0,0.00,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.066667,0.066667,0.066667,0.133333,0.133333,0.133333,0.000000,0.00,...,0.00,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011765,0.00,...,0.00,0.00,0.0,0.00,0.011765,0.000000,0.000000,0.011765,0.000000,0.011765
5,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.01,0.01,0.0,0.00,0.060000,0.000000,0.040000,0.010000,0.000000,0.000000
6,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Church and Wellesley,0.011628,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023256,0.00,...,0.00,0.00,0.0,0.00,0.000000,0.011628,0.011628,0.000000,0.011628,0.011628
8,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.00,...,0.00,0.00,0.0,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
9,"Design Exchange, Toronto Dominion Centre",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.00,...,0.00,0.00,0.0,0.01,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000


In [129]:
# Check the new size
downtown_grouped.shape

(18, 207)

## Print each neighbourhood with the top 5 venues

In [130]:
num_top_venues = 5

for hood in downtown_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.08
1                 Café  0.05
2           Steakhouse  0.04
3  American Restaurant  0.04
4                  Bar  0.04


----Berczy Park----
            venue  freq
0     Coffee Shop  0.07
1    Cocktail Bar  0.05
2  Farmers Market  0.04
3          Bakery  0.04
4            Café  0.04


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0    Airport Lounge  0.13
1   Airport Service  0.13
2  Airport Terminal  0.13
3          Boutique  0.07
4   Harbor / Marina  0.07


----Cabbagetown, St. James Town----
                venue  freq
0         Coffee Shop  0.09
1  Italian Restaurant  0.05
2         Pizza Place  0.05
3                Park  0.05
4                 Pub  0.05


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.15
1                Café  0.06
2  Italian Restaurant  0.05
3      

Create a pandas dataframe and display the top10 venues for each neighbourhood

In [131]:
#Function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Creating the new dataframe and display the top 10 venues for each neighbourhood

In [132]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = downtown_grouped['Neighbourhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,American Restaurant,Thai Restaurant,Asian Restaurant,Burger Joint,Restaurant,Hotel
1,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Café,Bakery,Steakhouse,Beer Bar,Seafood Restaurant,Cheese Shop,Beach
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Boutique,Bar,Harbor / Marina
3,"Cabbagetown, St. James Town",Coffee Shop,Pub,Italian Restaurant,Park,Bakery,Pizza Place,Restaurant,Café,Gift Shop,Japanese Restaurant
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Ice Cream Shop,Burger Joint,Sandwich Place,Chinese Restaurant,Spa,Gym / Fitness Center,Indian Restaurant


## Cluster Neighbourhoods

In [134]:
# Run k-means to cluster the neighbourhoods into 5 clusters
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighbourhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)
# Check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 3, 0, 0, 4, 2, 0, 0, 0], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighbourhood

In [135]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = toronto_data
# Merge the data
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
downtown_merged.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Playground,Trail,Building,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,0,Coffee Shop,Pub,Italian Restaurant,Park,Bakery,Pizza Place,Restaurant,Café,Gift Shop,Japanese Restaurant
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Gastropub,Hotel,Pub,Mediterranean Restaurant,Men's Store
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0,Coffee Shop,Park,Café,Bakery,Mexican Restaurant,Breakfast Spot,Restaurant,Pub,Theater,Brewery
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Fast Food Restaurant,Bakery,Sporting Goods Shop,Ramen Restaurant,Bookstore,Pizza Place


 Now let us visualize the resulting cluster

In [138]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], 
                                  downtown_merged['Neighbourhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [139]:
#examine the clusters
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, 
                     downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,0,Coffee Shop,Pub,Italian Restaurant,Park,Bakery,Pizza Place,Restaurant,Café,Gift Shop,Japanese Restaurant
2,Downtown Toronto,0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Gastropub,Hotel,Pub,Mediterranean Restaurant,Men's Store
3,Downtown Toronto,0,Coffee Shop,Park,Café,Bakery,Mexican Restaurant,Breakfast Spot,Restaurant,Pub,Theater,Brewery
4,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Fast Food Restaurant,Bakery,Sporting Goods Shop,Ramen Restaurant,Bookstore,Pizza Place
5,Downtown Toronto,0,Café,Coffee Shop,Restaurant,Hotel,Italian Restaurant,Beer Bar,Breakfast Spot,Bakery,Cosmetics Shop,Cocktail Bar
6,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Farmers Market,Café,Bakery,Steakhouse,Beer Bar,Seafood Restaurant,Cheese Shop,Beach
7,Downtown Toronto,0,Coffee Shop,Café,Italian Restaurant,Ice Cream Shop,Burger Joint,Sandwich Place,Chinese Restaurant,Spa,Gym / Fitness Center,Indian Restaurant
8,Downtown Toronto,0,Coffee Shop,Café,Bar,Steakhouse,American Restaurant,Thai Restaurant,Asian Restaurant,Burger Joint,Restaurant,Hotel
9,Downtown Toronto,0,Coffee Shop,Hotel,Aquarium,Café,Scenic Lookout,Fried Chicken Joint,Brewery,History Museum,Music Venue,Sports Bar
10,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Bar,Deli / Bodega,Gastropub,Italian Restaurant,Beer Bar


In [141]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, 
                     downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Park,Playground,Trail,Building,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop


In [142]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, 
                     downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Downtown Toronto,2,Grocery Store,Café,Park,Baby Store,Bank,Diner,Restaurant,Nightclub,Coffee Shop,Convenience Store


In [143]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, 
                     downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,3,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Boutique,Bar,Harbor / Marina


In [144]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 4, 
                     downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Downtown Toronto,4,Café,Bookstore,Restaurant,Sandwich Place,Japanese Restaurant,Italian Restaurant,Theater,Bar,Bakery,Dessert Shop
13,Downtown Toronto,4,Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Bar,Bakery,Vietnamese Restaurant,Dumpling Restaurant,Coffee Shop,Mexican Restaurant,Farmers Market
